<center><h1> GEARS Dataset </h1></center>

## Context

This notebook intend to load the dataset used in GEARS and in the benchmarking paper [\[1\]](#benchmark)

In particular, we will be using the following [datasets](#datasets)

## Datasets

**Raw**

- [Adamson](https://dataverse.harvard.edu/file.xhtml?fileId=6154417&version=3.0&toolType=PREVIEW): ht<span>tps://</span>dataverse.harvard.edu/api/access/datafile/6154417
- [Norman](https://dataverse.harvard.edu/file.xhtml?fileId=6154020&version=3.0&toolType=PREVIEW): ht<span>tps://</span>dataverse.harvard.edu/api/access/datafile/6154020

**Processed**

- [Replogle RPE1](https://dataverse.harvard.edu/file.xhtml?fileId=7458694&version=6.0&toolType=PREVIEW): ht<span>tps://</span>dataverse.harvard.edu/api/access/datafile/7458694
- [Relogle K562](https://dataverse.harvard.edu/file.xhtml?fileId=7458695&version=6.0&toolType=PREVIEW): ht<span>tps://</span>dataverse.harvard.edu/api/access/datafile/7458695
- [Norman scFoundation](https://figshare.com/articles/dataset/scFoundation_Large_Scale_Foundation_Model_on_Single-cell_Transcriptomics_-_processed_datasets/24049200?file=44477939): ht<span>tps://</span>figshare.com/ndownloader/files/44477939
## References

1. Ahlmann-Eltze, Huber, and Anders, “Deep Learning-Based Predictions of Gene Perturbation Effects Do Not yet Outperform Simple Linear Baselines.”<a id="benchmark"></a>


In [1]:
from pathlib import Path

import anndata as ad

from fine_tune.scripts import zip_data_download_wrapper

In [2]:
dataverse_url = "https://dataverse.harvard.edu/api/access/datafile/"
figshare_url = "https://figshare.com/ndownloader/files/"
file_map = {
    "adamson": {
        "host": dataverse_url,
        "file": "6154417"
    },
    "norman": {
        "host": dataverse_url,
        "file": "6154020"
    },
    "replogle_rpe1": {
        "host": dataverse_url,
        "file": "7458694"
    },
    "replogle_k562": {
        "host": dataverse_url,
        "file": "7458695"
    },
    "norman_sc_foundation": {
        "host": figshare_url,
        "file": "44477939"
    }
}
data_path = Path("./../../datasets")

for key, value in file_map.items():
    zip_data_download_wrapper(
        url=value["host"] + value["file"],
        save_path=data_path / ".zip" / key,
        data_path=data_path
    )

Found local copy...
Found local copy...
Found local copy...
Extracting zip file...
Done!
Found local copy...
Extracting zip file...
Done!
Found local copy...


In [ ]:
ad.read_h5ad()